In [1]:
#This file deals with high vs low sugar flies
from __future__ import print_function
import scipy.io as spio
import scipy.interpolate as spin
from scipy.linalg import norm
from numpy import *
from scipy.spatial.distance import cdist
import sys
import os
import cPickle as pickle
import pandas as pd
import pylab
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu

In [25]:
def index(x_pos,y_pos): # latest index
    rmin=20
    x_nor=x_pos-x_pos[0]
    y_nor=y_pos-y_pos[0]
    r=sqrt(x_nor**2+y_nor**2);
    
    vx=gradient(x_nor)
    vy=gradient(y_nor)
    v=sqrt(vx**2+vy**2)
    
    cutv= (v>0.00) 
    cutr= (r>rmin)
    cuts=cutv & cutr
    
    d=rmin+cumsum(sqrt(vx**2+vy**2)*cuts)
    

    #velocity calculation

    
    ind=r/d 
    ind[~cuts]=1
    indx=average(1-ind)
    if max(r)<50:
        return(0)
    return indx
    

def staytime(x_pos,y_pos): #Incomplete
    x_nor=x_pos-x_pos[0]
    y_nor=y_pos-y_pos[0]

    d=cumsum(sqrt(gradient(x_nor)**2+gradient(y_nor)**2))
    r=sqrt(x_nor**2+y_nor**2)
    
    #Calculating the nhoodtime/stop time
    points=dstack((x_pos,y_pos))[0]
    M=cdist(points,points)
    
    nmask=ones(x_pos.size,dtype=bool)

    i=0
    
    while(i<x_pos.size):
        bg=i
        end=min(x_pos.size,i+15)
        if max(M[i,i:end])<10:
            nmask[i:end]=False
            i=end+1
        else:
            i=i+1
# Different cuts
#    cuts= (v>0.1) & (d>5.0)
    

    stoptime=(x_pos.size)-(x_pos[cuts]).size   
    staytime=x_pos.size
    if len(where(r>20)[0])>0:
        staytime=min(where(r>20)[0])
    

In [26]:
#for wild type
rlist=[]
indexlist=[]
timelist=[]
filenamelist=[]
dt=1/50
count=0

#['6','12','11','14','15'],['1','5','9','10']
for mode in ['sugar','water']:
    rlist=[]
    indexlist=[]
    timelist=[]
    filenamelist=[]
    dt=1/50
    count=0
    for dirname,dirnames,filenames in os.walk("/home/pallab/Dropbox/work/Bio/Axel/Data/Satoshi/starvation without water/"+mode):
        for filename in filenames:
            if filename[-1]!='t':
                continue
            filenamelist.append(filename)
            data=spio.loadmat(os.path.join(dirname, filename))
            timestamps=ndarray.flatten(data['timestamps'])

            for key in data.keys():
                if key not in ['__globals__','__version__','__header__']:
                    exec(key+"=ndarray.flatten(data['"+key+"'])")

            #smoothing
            times=timestamps[0:x_pos.size]
            splx=spin.UnivariateSpline(times,x_pos,s=20)
            sply=spin.UnivariateSpline(times,y_pos,s=20)
            x_smooth=splx(times)
            y_smooth=sply(times)

    #        x_pos,y_pos=x_smooth,y_smooth
            x_nor=x_pos-x_pos[0]
            y_nor=y_pos-y_pos[0]

            r=sqrt(x_nor**2+y_nor**2)


        #x_smooth=splx(times)
        #y_smooth=sply(times)




            timelist.append(x_pos.size)

            dt=1/30.0
        #    print("%s %0.3f %d %d"%(filename,index,stoptime*dt,staytime*dt),end='\n')
            rlist.append(average(r))
            indx=index(x_pos,y_pos)
            indexlist.append(indx)
            print(filename,average(r),indx,end='\n')




            plt.plot(x_nor, y_nor, 'g-')

            # location of dots
            plt.xlim([-500,500]) #[200,1100] for Tachykinin
            plt.ylim([-500,500])  #[75,1000] for Tachykinin
            plt.axes().set_aspect('equal')

    #        plt.plot(x_pos[0],y_pos[0],'ko',ms=15, mfc='none', mew=1)        

            plt.savefig("/home/pallab/AxelWork/pix/"+str.split(filename,".")[0]+'.png')
            plt.close()
            plt.show()
    #idxs=array([round(indexlist[keys],2) for keys in indexlist.keys()])
    exec("rlist_%s=rlist"%mode)
    exec("timelist%s=timelist"%mode)
    exec("id_%s=indexlist"%mode)

11.mat 102.65469404 0.866599653698
18.mat 87.192422045 0.770601818418
14.mat 29.5026930706 0.818880492
1.mat 51.4136049649 0.8762960069
8.mat 44.1755139608 0.868883673151
17.mat 59.4051984835 0.762657739788
6.mat 92.6810857485 0.730207650056
16.mat 48.7067808053 0.71872717679
20.mat 92.0051782931 0.733352228994
9.mat 67.8120639939 0.747996086747
19.mat 92.0949519125 0.555291996437
10.mat 40.0668462147 0.029872635506
3.mat 70.3294804606 0.431724848301
4.mat 147.772456176 0.831014760241
15.mat 90.3426086683 0.70403507806
7.mat 88.8622022327 0.84761833336
5.mat 56.0139442169 0.18495358908
12.mat 74.4820497476 0.724952593532
13.mat 64.7520536136 0.374663420518
2.mat 42.3016205669 0.604667250575
11.mat 119.443252991 0.175508730375
18.mat 36.9116229671 0.0196281296129
14.mat 128.480950343 0.244643588027
1.mat 75.1152237627 0.13994706146
8.mat 49.6229318403 0.472545795728
17.mat 80.1631998244 0.122420373508
6.mat 49.5002753795 0.316486453691
16.mat 60.0929130471 0.0451276137267
20.mat 66.9474

In [42]:
#Box plots of the data

plt.xlim(0.5,4.0)
plt.ylim(0,1.0)

data=[id_water,id_sugar]
#plt.boxplot(data)

bp = plt.boxplot(data, widths=0.5)
plt.setp(bp['boxes'], color='black') 
#for i in range(2):
y = data[0]
x = random.normal(1+0, 0.04, size=len(y))
plt.plot(x, y, 'g.', alpha=0.6, ms=11)

y = data[1]
x = random.normal(1+1, 0.04, size=len(y))
plt.plot(x, y, 'r.', alpha=0.6, ms=11)
plt.axes().set_aspect(4.0)
plt.show()

In [35]:
mannwhitneyu(id_sugar,id_water)

(34.0, 3.7886905298631322e-06)

NameError: name 'perasonr' is not defined

In [5]:
from scipy.stats.stats import pearsonr

In [28]:
pearsonr(id_water,timelistwater)

(0.28239716194700926, 0.22768451050961988)

In [43]:
plt.plot(timelistwater,id_water,"go")
plt.plot(timelistsugar,id_sugar,"ro")
plt.xlim([0,4000])
plt.ylim([0,1.0])
plt.show()

In [36]:
pearsonr(id_sugar,timelistsugar)

(0.57510852483638986, 0.0079819967407856931)

In [45]:
pearsonr(id_water,timelistwater)

(0.28239716194700926, 0.22768451050961988)

In [44]:
map(lambda x:round(x,2),id_sugar)